# Part 15

# Using Code Interpreter

Universal code for the entire notebook

In [11]:
# Uncomment the line below to make sure you have all the packages needed
# %pip install -r requirements.txt

In [12]:
# Import necessary libraries
from openai import OpenAI  # Used for interacting with OpenAI's API
from typing_extensions import override  # Used for overriding methods in subclasses
from openai import AssistantEventHandler  # Used for handling events related to OpenAI assistants
from IPython.display import display, Markdown, clear_output

import time
import threading

In [13]:
# Create an instance of the OpenAI class to interact with the API.
# This assumes you have set the OPENAI_API_KEY environment variable.
client = OpenAI() 

In [14]:
class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize the results list

    @override
    def on_text_created(self, text) -> None:
        """Handle the event when text is first created."""
        print("\nassistant text > ", end="", flush=True)
        self.results.append(text)

    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        print(delta.value, end="", flush=True)
        self.results.append(delta.value)

    def on_tool_call_created(self, tool_call):
        """Handle the event when a tool call is created."""
        print(f"\nassistant tool > {tool_call.type}\n", flush=True)

    def on_tool_call_delta(self, delta, snapshot):
        """Handle the event when there is a delta (update) in a tool call."""
        if delta.type == 'code_interpreter':
            if delta.code_interpreter.input:
                print(delta.code_interpreter.input, end="", flush=True)
                self.results.append(delta.code_interpreter.input)
            if delta.code_interpreter.outputs:
                print("\n\noutput >", flush=True)
                for output in delta.code_interpreter.outputs or []:  # Adding a safeguard
                    if output.type == "logs":
                        print(f"\n{output.logs}", flush=True)
                        self.results.append(output.logs)


## Creating an Assistant with Code Interpreter Enabled

Our first step is to create an Assistant that can use Code Interpreter

In [15]:
# Create an assistant using the client library.
assistant = client.beta.assistants.create(
    model="gpt-4o",  # Specify the model to be used.
    
    instructions=""" 
        You are a helpful assistant.
    """,
    
    name="Code Interpreter Assistant",  # Give the assistant a name.
    
    tools=[{"type": "code_interpreter"}], # Add the code interpreter capability to the assistant.
    
    metadata={  # Add metadata about the assistant's capabilities.
        "can_be_used_for_code_analysis": "True",
        "can_do_python": "True",
    },
    temperature=1,  # Set the temperature for response variability.
    top_p=1,  # Set the top_p for nucleus sampling.
)

# Print the details of the created assistant to check its properties.
print(assistant)  # Print the full assistant object.
print("\n\n")
print(assistant.name)  # Print the name of the assistant.
print(assistant.metadata)  # Print the metadata of the assistant.

Assistant(id='asst_nZpzQEtLhmpyx15fI44VjqVH', created_at=1718456112, description=None, instructions=' \n        You are a helpful assistant.\n    ', metadata={'can_be_used_for_code_analysis': 'True', 'can_do_python': 'True'}, model='gpt-4o', name='Code Interpreter Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)



Code Interpreter Assistant
{'can_be_used_for_code_analysis': 'True', 'can_do_python': 'True'}


## Passing Files to Code Interpreter

There are a variety of ways to get files for Code Interpreter to use. 
- Assistant files - viewable by all runs that use the assistant.
- Thread files - only viewable by runs that use the thread. 

Let's review the code for the two main approaches.

### Getting Files to the Assistant

First, you have to have a file that has been uploaded so we can pass it to our assistant.

In [16]:
# Upload a file with an "assistants" purpose
assistant_file = client.files.create(
    file=open("./artifacts/penguins_size.csv", "rb"),
    purpose='assistants'
)

print(assistant_file)

FileObject(id='file-XTz2oAJhhoIAdmKZiEzZbV6a', bytes=13519, created_at=1718456112, filename='penguins_size.csv', object='file', purpose='assistants', status='processed', status_details=None)


Next, we need to modify our Assistant with the new file information. 

In [17]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
            "file_ids": [assistant_file.id]
        }
    }
)

print(assistant)

Assistant(id='asst_nZpzQEtLhmpyx15fI44VjqVH', created_at=1718456112, description=None, instructions=' \n        You are a helpful assistant.\n    ', metadata={'can_be_used_for_code_analysis': 'True', 'can_do_python': 'True'}, model='gpt-4o', name='Code Interpreter Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-XTz2oAJhhoIAdmKZiEzZbV6a']), file_search=None), top_p=1.0)


Finally, let's run a message and see if it is working.

In [18]:

# Your assistant code remains unchanged
assistant_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Give me a summary of the file penguins_size.csv."
        },
    ]
)

with client.beta.threads.runs.stream(
    thread_id=assistant_thread.id,
    assistant_id=assistant.id,
    instructions="""
    You are a helpful assistant.
    """,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()



assistant tool > code_interpreter

import pandas as pd

# Load the uploaded CSV file
file_path = '/mnt/data/file-XTz2oAJhhoIAdmKZiEzZbV6a'
penguins_data = pd.read_csv(file_path)

# Display a summary of the data
summary = penguins_data.describe(include='all')
summary
assistant text > The `penguins_size.csv` file contains data on penguins with the following columns:

1. **species**: The species of the penguin (Adelie, Gentoo, Chinstrap). There are 3 unique species in the dataset.
2. **island**: The island where the penguin was found (Biscoe, Dream, Torgersen). There are 3 unique islands in the dataset.
3. **culmen_length_mm**: The length of the culmen (bill) in millimeters.
   - Count: 342
   - Mean: 43.92 mm
   - Standard Deviation: 5.46 mm
   - Min: 32.1 mm
   - Max: 59.6 mm
4. **culmen_depth_mm**: The depth of the culmen (bill) in millimeters.
   - Count: 342
   - Mean: 17.15 mm
   - Standard Deviation: 1.97 mm
   - Min: 13.1 mm
   - Max: 21.5 mm
5. **flipper_length_mm**: The length 

### Formatting the Output
What if we want to format the markdown output? There are two ways to do it. The easy way is to just let the output render without streaming and format it.

In [19]:
# Need a thread to send message and get output
assistant_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Give me a summary of the file penguins_size.csv."
        },
    ]
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=assistant_thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=assistant_thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

display(Markdown(message_content.value))


The file `penguins_size.csv` contains data on various physical measurements of penguins. Here is a summary of the dataset:

### Shape
- The dataset contains 344 entries (rows) and 7 columns.

### Columns
- `species`: Type of penguin species (categorical)
- `island`: Location/island where the penguin was observed (categorical)
- `culmen_length_mm`: Length of the bill in millimeters (numerical)
- `culmen_depth_mm`: Depth of the bill in millimeters (numerical)
- `flipper_length_mm`: Length of the flipper in millimeters (numerical)
- `body_mass_g`: Weight of the penguin in grams (numerical)
- `sex`: Gender of the penguin (categorical)

### Information
- `culmen_length_mm`, `culmen_depth_mm`, `flipper_length_mm`, and `body_mass_g` columns have 342 non-null values (indicating missing data in 2 entries).
- `sex` column has 334 non-null values (indicating missing data in 10 entries).

### Descriptive Statistics
- **culmen_length_mm**:
  - Mean: 43.92 mm
  - Std Dev: 5.46 mm
  - Min: 32.1 mm
  - Max: 59.6 mm
  
- **culmen_depth_mm**:
  - Mean: 17.15 mm
  - Std Dev: 1.97 mm
  - Min: 13.1 mm
  - Max: 21.5 mm
  
- **flipper_length_mm**:
  - Mean: 200.92 mm
  - Std Dev: 14.06 mm
  - Min: 172 mm
  - Max: 231 mm
  
- **body_mass_g**:
  - Mean: 4201.75 g
  - Std Dev: 801.95 g
  - Min: 2700 g
  - Max: 6300 g

The "hard" way is to stream the output and update the display while streaming to show the formatted text. This is what ChatGPT does when you use it. This will require modifying our event handler to be more streamlined, formatting output, and updating our display.

In [ ]:
class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize the results list

    
    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        print(delta.value, end="", flush=True)
        self.results.append(delta.value)


# Your assistant code remains unchanged
assistant_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Give me a summary of the file penguins_size.csv."
        },
    ]
)

with client.beta.threads.runs.stream(
    thread_id=assistant_thread.id,
    assistant_id=assistant.id,
    instructions="""
    You are a helpful assistant.
    """,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

In [24]:
class EventHandler(AssistantEventHandler):
    """Custom event handler for processing assistant events."""

    def __init__(self):
        super().__init__()
        self.results = []  # Initialize the results list

    
    @override
    def on_text_delta(self, delta, snapshot):
        """Handle the event when there is a text delta (partial text)."""
        print(delta.value, end="", flush=True)
        self.results.append(delta.value)


# Your assistant code remains unchanged
assistant_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Give me a summary of the file penguins_size.csv."
        },
    ]
)

with client.beta.threads.runs.stream(
    thread_id=assistant_thread.id,
    assistant_id=assistant.id,
    instructions="""
    You are a helpful assistant.
    """,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


Let's start by loading the contents of the file "penguins_size.csv" to examine its structure and understand its contents. I will read the file and provide a summary.

Let's begin by loading and inspecting the data.The "penguins_size.csv" dataset consists of 344 entries (or rows) and 7 columns. Here's a brief summary of the columns:

1. **species**: The species of the penguin (categorical data).
2. **island**: The island where the penguin was found (categorical data).
3. **culmen_length_mm**: The length of the penguin's culmen (bill) in millimeters (continuous data).
4. **culmen_depth_mm**: The depth of the penguin's culmen (bill) in millimeters (continuous data).
5. **flipper_length_mm**: The length of the penguin's flipper in millimeters (continuous data).
6. **body_mass_g**: The body mass of the penguin in grams (continuous data).
7. **sex**: The sex of the penguin (categorical data).

### Data Information:
- There are some missing values (`NaN`), particularly in the columns `culmen_

### Getting Files to the Thread

First, we need a file uploaded.


In [ ]:
# Upload a file with an "assistants" purpose
thread_file = client.files.create(
    file=open("./artifacts/daily-bike-share.csv", "rb"),
    purpose='assistants'
)

print(thread_file)

Second, we need a thread to attach the file to

In [ ]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Give me a summary of the daily-bike-share.csv file."
        },
    ]
)

print(thread)

Third, we can update the thread with the file information

In [ ]:
updated_thread = client.beta.threads.update(
    thread_id=thread.id,
    tool_resources={
        "code_interpreter": {
            "file_ids": [thread_file.id]
        }
    }
)

print(updated_thread)

Finally, let's run it against a new assistant and see the results

In [ ]:
# Create an assistant using the client library.
thread_assistant = client.beta.assistants.create(
    model="gpt-4o",  # Specify the model to be used.
    
    instructions=""" 
        You are a helpful assistant.
    """,
    
    name="Code Interpreter Assistant Using Thread Data",  # Give the assistant a name.
    
    tools=[{"type": "code_interpreter"}], # Add the code interpreter capability to the assistant.
    
    metadata={  # Add metadata about the assistant's capabilities.
        "can_be_used_for_code_analysis": "True",
        "can_do_python": "True",
    },
    temperature=1,  # Set the temperature for response variability.
    top_p=1,  # Set the top_p for nucleus sampling.
)

# Print the details of the created assistant to check its properties.
print(assistant)  # Print the full assistant object.
print("\n\n")
print(assistant.name)  # Print the name of the assistant.
print(assistant.metadata)  # Print the metadata of the assistant.

In [ ]:
# stream the output from the assistant
with client.beta.threads.runs.stream(
    thread_id=updated_thread.id,
    assistant_id=thread_assistant.id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

### Text Results from files in Assistants and Threads

Let's see what happens if we use an assistant with a file and a thread with a file together

In [ ]:
super_thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Give me a summary of the penguins-size.csv and daily-bike-share.csv files."
        },
    ]
)

print(super_thread)

In [ ]:
super_updated_thread = client.beta.threads.update(
    thread_id=super_thread.id,
    tool_resources={
        "code_interpreter": {
            "file_ids": [thread_file.id]
        }
    }
)

print(super_updated_thread)

In [ ]:
# stream the output from the assistant
with client.beta.threads.runs.stream(
    thread_id=super_updated_thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

### Full Results Output

We may have other things that are produced

In [ ]:
import openai
from IPython.display import Image, display
import json

# Initialize the OpenAI client
client = openai.OpenAI()

# Upload the file
file = client.files.create(
    file=open("./artifacts/penguins_size.csv", "rb"),
    purpose='assistants'
)

# Create the assistant with the Code Interpreter tool
assistant = client.beta.assistants.create(
    instructions="You are a data analyst. When provided with a file, analyze the data and generate visualizations.",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
        "code_interpreter": {
            "file_ids": [file.id]
        }
    }
)

# Create a thread to start the analysis
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Please analyze the data in the uploaded file and generate relevant visualizations.",
            "attachments": [
                {
                    "file_id": file.id,
                    "tools": [{"type": "code_interpreter"}]
                }
            ]
        }
    ]
)

# Fetch the results of the analysis
response = client.beta.threads.retrieve(thread.id)

# Convert the response to a dictionary to access its attributes
response_dict = response.to_dict()

# Debug: Print the entire response to understand its structure
print(json.dumps(response_dict, indent=2))

# Access messages from the thread
messages = response_dict.get('messages', [])
for message in messages:
    content = message.get('content', '')
    print(content)
    
    # If the response includes images, download and save them
    if "image_file" in message:
        image_file_id = message["image_file"]["file_id"]
        image_data = client.files.content(image_file_id)
        with open("analysis_image.png", "wb") as image_file:
            image_file.write(image_data.read())
        # Display the image in the notebook
        display(Image("analysis_image.png"))
